In [2]:

from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(openai_api_key=api_key,temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

invoke_chain("제 이름은 담입니다.")


content='안녕하세요, 담님. 무엇을 도와드릴까요?'


In [3]:
invoke_chain("저는 서울에 삽니다.")


content='서울은 아름다운 도시입니다. 서울에서 즐길 만한 장소나 활동이 있나요? 혹시 도움이 필요하시면 언제든지 물어보세요.'


In [4]:
invoke_chain("제 이름이 뭘까요?")


content='죄송합니다, 제가 당신의 이름을 알 수는 없습니다. 하지만 당신이 "Dan"이라고 소개하셨죠. 어떤 도움이 필요하신가요, Dan님?'
